In [71]:
from system import *
s1 = system('x-o-o',MMA=True,ManyVariables=False,TwoPhotonResonance= False)
#s1 = load_object('o')

Initializing system x-o-o  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  x-o-o  initialized in 16.9 seconds.


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [72]:
diag = 1  #input n corresponding to diag element

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('dct',domain='positive',  latex_name =r'\tilde{\delta }_c')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')

#(gamma_f=gamma)#gamma_g=0,

#Take only one of the compl conj terms. Switch maybe second 0 to 1
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage

    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)
    #var('n')
    #b = b.subs(DE=DEg*gamma)
    #b = b.subs(De=Deg*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full= False)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b

show( (symround(h_eff,digits=1)))

-((3*Det*dct - C)*c^2 - (-2*I*Det*dct + 1.5*I*C)*c*dct - (0.25*Det*dct - 0.25*C)*dct^2)/((Det*c^2 + 1.5*I*Det*c*dct - 0.25*Det*dct^2 - I*C*c + 0.25*C*dct)*C*R_f - ((3*Det*dct - C)*c^2 - (-2*I*Det*dct + 1.5*I*C)*c*dct - (0.25*Det*dct - 0.25*C)*dct^2)*DEt)

In [60]:
factor = 1#*16*4


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(h_eff.numerator().expand()),full=True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(h_eff.denominator().expand()),full= not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


-2*Det*c*dct - 0.500*I*Det*dct^2 + C*c + 0.500*I*C*dct



 Latex
-2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} - 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} + C {c} + 0.500i \, C {\tilde{\delta }_c}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


(Det*c + 0.500*I*Det*dct - 0.500*I*C)*C*R_f - (2*Det*c*dct + 0.500*I*Det*dct^2 - C*c - 0.500*I*C*dct)*DEt



 Latex
{\left({\tilde{\Delta }_e} {c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c} - 0.500i \, C\right)} C R_{f} - {\left(2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} - C {c} - 0.500i \, C {\tilde{\delta }_c}\right)} {\tilde{\Delta }_E}


# Expand linearly in the limit:   $C\rightarrow \infty$

In [61]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)
#t = q._mathematica_().Series((x,0,2))
#t_exp = t[3]
#t_st = t[4]
#tayl = 0
#for i in range(1,len(t_exp)):
#    tayl += MMA_simplify( t_exp[i]._sage_() ) / C**(i-1+Integer(t[4]))

show(symround(tayl))

#with sage
#tayl = b.taylor(C_c,infinity, 1)
#show(symround(MMA_simplify(tayl)))

((2*I*R_v*c_k + 0.66*I)*R_v*c_k + 0.042*I)/(((2*I*DE + 1.3*R_f)*R_v^2*c_k^2 + (0.66*I*DE + R_f)*R_v*c_k + 0.042*I*DE + 0.083*R_f)*C)

# Lindblau Operators

Choose which of the Lindblau operators to view and which non zero element of it

In [62]:
def lind(lind_op,which):    
    L_matrix_orig = s1.lindblau_list[lind_op]
    L_matrix      = s1.eff_lindblau_list[lind_op] #lindblau_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
    #L_elem = L_elem.subs(DE=gamma*DEg)
    #L_elem = L_elem.subs(De=gamma*Deg)                     
    #COUPLINGS
    b = L_elem
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    
    L_elem = b
    
  
    L_elem = L_elem._mathematica_().PowerExpand()._sage_()
    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))

    

    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])
    return L_elem


# Expand linearly in the limit:   $C\rightarrow \infty$

In [22]:
#with mathematica

q = L_elem.subs(C=1/x)
q = MMA_simplify(q)
t = q._mathematica_().Series((x,0,1)).Normal()

tayl = t._sage_().subs(x=1/C)
tayl = SR(str(tayl).replace('Sqrt','sqrt'))
show(symround(tayl))

Omega*sqrt(kappa_c)/(sqrt(C*gamma*kappa_c)*sqrt(R_f))

In [63]:
lind_op = 0  #which lindblau operator
which   = 0  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


sqrt(C)*Omega*sqrt(R_f)/((C*R_f - DEt*dct)*sqrt(gamma))

 |0f00> < 0g00 | 

 Original coefficient :


sqrt(kappa_c)

\frac{\sqrt{C} {\Omega} \sqrt{R_{f}}}{{\left(C R_{f} - {\tilde{\Delta }_E} {\tilde{\delta }_c}\right)} \sqrt{{\gamma}}}


In [40]:
lind_op = 0  #which lindblau operator
which   = 0  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


sqrt(C*gamma*kappa_c)*Omega*sqrt(R_f)/((C*R_f*gamma + 0.50*I*DEt*gamma)*sqrt(kappa_c))

 |0f00> < 0g00 | 

 Original coefficient :


sqrt(kappa_c)

\frac{\sqrt{C {\gamma} {\kappa_c}} {\Omega} \sqrt{R_{f}}}{{\left(C R_{f} {\gamma} + 0.50i \, {\tilde{\Delta }_E} {\gamma}\right)} \sqrt{{\kappa_c}}}


In [14]:
factor = 8


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


(Det*(16*I*c + 4*I) + 8*C)*C*Omega*sqrt(R_f)*sqrt(kappa_c)



 Latex
{\left({\tilde{\Delta }_e} {\left(16 i \, {c} + 4 i\right)} + 8 \, C\right)} C {\Omega} \sqrt{R_{f}} \sqrt{{\kappa_c}}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


((Det*(16*I*c + 4*I) + 8*C)*C*R_f - (2*Det*(8*c + 1) - C*(16*I*c + 4*I))*DEt)*sqrt(C)*sqrt(gamma)*sqrt(kappa_c)



 Latex
{\left({\left({\tilde{\Delta }_e} {\left(16 i \, {c} + 4 i\right)} + 8 \, C\right)} C R_{f} - {\left(2 \, {\tilde{\Delta }_e} {\left(8 \, {c} + 1\right)} - C {\left(16 i \, {c} + 4 i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{C} \sqrt{{\gamma}} \sqrt{{\kappa_c}}


In [28]:
show(symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3) / (symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3)))

32*sqrt(C)*Det^2*Omega*sqrt(R_f)*c^2*e^(2*I*phi)/(((2*(16*c^2 + 12*c + 1)*Det^2 - C*Det*(48*I*c + 8*I) - 8*C^2)*C*R_f + (4*(16*c^2 + 10*c + 1)*C*Det - (-48*I*c^2 - 16*I*c - I)*Det^2 - C^2*(16*I*c + 4*I))*DEt)*sqrt(gamma))

In [29]:
latex(symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3) / (symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3)))

\frac{32 \, \sqrt{C} {\tilde{\Delta }_e}^{2} {\Omega} \sqrt{R_{f}} {c}^{2} e^{\left(2 i \, {\phi}\right)}}{{\left({\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} {\tilde{\Delta }_e}^{2} - C {\tilde{\Delta }_e} {\left(48 i \, {c} + 8 i\right)} - 8 \, C^{2}\right)} C R_{f} + {\left(4 \, {\left(16 \, {c}^{2} + 10 \, {c} + 1\right)} C {\tilde{\Delta }_e} - {\left(-48 i \, {c}^{2} - 16 i \, {c} - i\right)} {\tilde{\Delta }_e}^{2} - C^{2} {\left(16 i \, {c} + 4 i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}}